In [20]:
from dotenv import load_dotenv
from langchain.embeddings import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
from langchain.vectorstores import FAISS

import os
import pickle

In [5]:
# Load index from file

In [6]:
load_dotenv()

True

In [8]:
loaded_faiss_vs = FAISS.load_local(
    folder_path="./vectordb/faiss/nfcorpus/",
    embeddings=OpenAIEmbeddings())

In [9]:
retriever = loaded_faiss_vs.as_retriever(search_kwargs={'k': 5})

In [10]:
# Define the RAG pipeline

In [13]:
model = 'gpt-3.5-turbo-instruct'
llm = OpenAI(model_name=model, openai_api_key=os.getenv("OPENAI_API_KEY"))

In [14]:
template = """Answer the question or Explain the topic given this additional context: {context}
Question: {question}"""

In [17]:
prompt = ChatPromptTemplate.from_template(template)

In [27]:
docs_file_path = './openai_embeddings/nfcorpus/doc_embeddings.pkl'
with open(docs_file_path, 'rb') as file:
    docs = pickle.load(file)
print("Document embeddings loaded successfully.")

Document embeddings loaded successfully.


In [18]:
def format_docs(_docs):
    ls = []
    for doc in _docs:
        if doc.page_content in docs:
            ls.append(docs[doc.page_content]["text"][:800])
    return ls

In [21]:
chain = ({"context": retriever | format_docs, "question": RunnablePassthrough()} 
         | prompt 
         | llm 
         | StrOutputParser())

In [ ]:
# Run the RAG pipeline

In [24]:
query_file_path = './openai_embeddings/nfcorpus/query_embeddings.pkl'
with open(query_file_path, 'rb') as file:
    loaded_queries = pickle.load(file)
print("Query embeddings loaded successfully.")

Query embeddings loaded successfully.


In [28]:
chain.invoke(loaded_queries["PLAIN-2"]["text"])

'\n\nBased on the additional context provided, it is not clear if cholesterol statin drugs directly cause breast cancer. There is some evidence that suggests that statins may decrease the risk of cancer, including breast cancer, but the available evidence is conflicting. Several studies have been conducted to investigate the association between statin use and breast cancer risk, but the results have been inconsistent and lacked the ability to evaluate long-term statin use. Some studies have even suggested that statins may have a protective effect against breast cancer recurrence or mortality. However, more research is needed to fully understand the relationship between cholesterol statin drugs and breast cancer. '